## Connect to the database

In [ ]:
from sqlalchemy import create_engine

servername = "osprey"
dbname = "NECPR"
engine = create_engine(
    "mssql+pyodbc://@"
    + servername
    + "/"
    + dbname
    + "?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server"
)
conn = engine.connect()

## List table names in database

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

table_names = []
for schema in schemas:
    for table_name in inspector.get_table_names(schema=schema):
        #for column in inspector.get_columns(table_name, schema=schema):
        #    print("Column: %s" % column)
        table_names.append(f"{schema}.{table_name}")
        print(f"{schema}.{table_name}")

## Write all the tables

In [ ]:
from os.path import join, exists
from os import mkdir
destination_directory = f"../temp/{dbname}"
if not exists(destination_directory):
    mkdir(destination_directory)

import polars as pl
import pandas as pd

for table in table_names:
    query = f"SELECT * FROM {table}"
    print(query)
    #df = pl.read_database(query=query, connection=conn, infer_schema_length =1000)
    #df.write_parquet(join(destination_directory, table + ".parquet"))
    if "\\" not in table:
        df = pd.read_sql(sql=query, con = conn)
        df.to_parquet(join(destination_directory, table.replace("\\","_") + ".parquet"))



In [ ]:
import polars as pl
from sqlalchemy import create_engine
servername = "osprey"
dbname = "GOV_REG_PAM"
conn = create_engine('mssql+pyodbc://@' + servername + '/' + dbname + '?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server')

query = "SELECT * FROM annexIX.PaMs"
df = pl.read_database(query=query, connection=conn.connect())
df.write_parquet(r"~/temp/python/PaMs.parquet")


df_from_parquet = pl.read_parquet(r"~/temp/python/table1.parquet")
df_from_parquet.head()

In [ ]:
df_from_parquet = pl.read_parquet(r"~/temp/python/table1.parquet")
df_from_parquet.head()

In [ ]:
df_from_parquet["2030"] > 55
#df.filter((pl.col("2030") <= 3) & (pl.col("2040") <= 3))

q1 = pl.col("Reporting_Element").str.contains("Role")
q2 = pl.col("2030") < 0





In [ ]:
import matplotlib.pyplot as plt
import numpy as np
a = df.filter(q1 | q2)["2030"]
plt.plot(a)

In [ ]:
import duckdb
ddb = duckdb.read_parquet(r"~/temp/python/table1.parquet")


In [ ]:
import polars as pl
import plotly.express as px

wide_df = pl.DataFrame(
    {
        "nation": ["South Korea", "China", "Canada"],
        "gold": [24, 10, 9],
        "silver": [13, 15, 12],
        "bronze": [9, 12, 12],
    }
).to_pandas()

fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()